Annotating and reading the Walsh lab data 

In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from matplotlib import pyplot as plt
import pandas as pd
import attila_utils
from bsmcalls import readVCF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%connect_info

{
  "shell_port": 33335,
  "iopub_port": 60373,
  "stdin_port": 43871,
  "control_port": 39897,
  "hb_port": 59009,
  "ip": "127.0.0.1",
  "key": "c54fa6a8-6b0c1fc237b6f59329c84c66",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-540498df-0849-49e1-9563-99466eedf0ec.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
